In [1]:
import numpy as np #scientific computing
import pandas as pd #data manipulation and analysis
import sys #used for handling command-line arguments
import re #used for working with patterns in strings

import os #to read env variables
import tqdm #progress bar
import time
import psycopg2 #used for interacting with the database
from sqlalchemy import create_engine
from io import StringIO
import csv
from dotenv import load_dotenv
from helpers import *
from postgres_helpers import *

/tmp/ipykernel_621649/1478519670.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd #data manipulation and analysis


In [4]:
load_dotenv()# write details like host, user, password, dbname in .env file, this loads them into this script
db_name = os.getenv("PG_DB_NAME")
user = os.getenv("PG_DB_USER")
password = os.getenv("PG_DB_PASSWORD")
host = os.getenv("PG_DB_HOST")
port = int(os.getenv("PG_DB_PORT"))

In [2]:
csv_file_path = '../processed'
num_duplicates = 30
all_dataframes = process_data(csv_file_path, num_duplicates)
print(len(all_dataframes))

2015


In [6]:

engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{db_name}')

startTime = time.time()
df_concat = pd.concat(all_dataframes)
df_concat.to_sql('model1', con=engine, if_exists='append', index=False)
engine.dispose()


endTime = time.time()
insertionTime = endTime - startTime
print("Time taken to insert using to_sql: ", insertionTime)

Time taken to insert using to_sql:  209.08628797531128


In [9]:
 #"SELECT (*) FROM model2 LIMIT 1",
queries = [
    "SELECT COUNT(*) FROM model1",
    "SELECT COUNT(*) FROM model1 WHERE \"Zygosity\" = 'Homozygous'",
    "SELECT COUNT(*) FROM model1 WHERE \"Zygosity\" = 'Heterozygous' AND \"Gene\" LIKE 'PADI4'",
    "SELECT COUNT(*) FROM model1 WHERE \"Gene\" LIKE 'PADI4'",
]
for query in queries:
    print(query)
    result, timeTaken = runQ(query,postgresQ)
    print(result)
    print("Time for this query was: ", timeTaken*1000, "ms")

SELECT COUNT(*) FROM model1
[(526783,)]
Time for this query was:  53.77531051635742 ms
SELECT COUNT(*) FROM model1 WHERE "Zygosity" = 'Homozygous'
[(86614,)]
Time for this query was:  63.73739242553711 ms
SELECT COUNT(*) FROM model1 WHERE "Zygosity" = 'Heterozygous' AND "Gene" LIKE 'PADI4'
[(1457,)]
Time for this query was:  65.18077850341797 ms
SELECT COUNT(*) FROM model1 WHERE "Gene" LIKE 'PADI4'
[(1984,)]
Time for this query was:  59.91029739379883 ms
